# Object-based Image Classification
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Generate initial segmentation](#initialsegm)
* [Image labelling](#label)
* [Training and test set generation](#trainset)
* [Random forest classifier](#randomforest)
* [Parameter tuning and feature selection](#featureselect)
* [Comparison with other classifiers](#comparison)


<a id="intro"></a> 
## Introduction
Segmentation using classical computer vision approaches such as watershed may produce results that have to be filtered based on their region properties to eliminate segmentation artefacts, such as small objects, noise, etc. If the image set is large (such as in high-throughput screening), filtering based on fixed thresholds may be supoptimal. In order to automate the task of filtering artefacts we can resort to machine learning approaches. 


There is a number of different schemes and machine learning models that can be used for this purpose. Here we will show how to train an object-based random forest classifier. The input for this classifier will be cropped bounding regions of the initial segmentation generated by simple spot detection. The task will be to classify the image patches into various cell types. 

Here we are dealing with coculture images with 2 cell types: stroma and leukemia cells, which were not stained differentially. Due to this minimal staining palette we need to use machine learning to automate the process of identification of leukemia cells.

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

import sys
sys.path.append('..')

javabridge.start_vm(class_path=bf.JARS)

In [ ]:
from base.utils import load_imgstack
imgstack = load_imgstack(fname="data/AML_trainset/180528_Plate3/r02c14.tiff")

In [ ]:
# remove a 'dummy' z-axis
img = np.squeeze(imgstack)

In [ ]:
from base.plot import plot_channels
gamma = 0.4
plot_channels([img[:,:,i]**gamma for i in range(3)],
              nrow=1, ncol=3, cmap='gray',
              titles=['Hoechst', 'Lysosomal dye', 'Viability'])

In [ ]:
from base.plot import combine_channels

img_rgb = combine_channels([img[:,:,i] for i in range(3)],
                            colors=['blue', 'red', 'green'],
                            blend=[1.5,1.5,2],
                            gamma=[0.6, 0.6,0.6])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_rgb)
plt.axis('off')

<a id="initialsegm"></a> 
## Generate initial segmentation
We can generate initial segmentation using spot detection in the nucleus channel:

In [ ]:
from transform.process import threshold_img
hoechst = img[:,:,0]**gamma
img_th = threshold_img(hoechst, method='otsu')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_th)
plt.axis('off')

In [ ]:
from skimage.morphology import binary_erosion, disk
img_th = binary_erosion(threshold_img(hoechst, binary=True, method='otsu'), disk(5))

In [ ]:
from skimage.measure import label
from skimage.color import label2rgb
segm = label(img_th, connectivity=1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(label2rgb(segm, image=hoechst, bg_label=0))
ax.axis('off')

In [ ]:
from skimage.measure import regionprops
feats =  regionprops(label_image=segm, intensity_image=hoechst)

In [ ]:
def get_feattable(feats, keys):
    return pd.DataFrame({key: [f[key] for f in feats] for key in keys})

In [ ]:
feat_df = get_feattable(feats, keys=['area', 'eccentricity', 'mean_intensity', 'perimeter'])

In [ ]:
upper = np.logical_and(feat_df.area < 6000, feat_df.perimeter < 1000)
lower = np.logical_and(feat_df.area > 100, feat_df.perimeter > 50)
feat_subset = np.logical_and(lower, upper)
# label count of non-background objects starts with 1
label_subset = np.where(feat_subset)[0] + 1

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(label2rgb(segm*np.isin(segm, label_subset), image=hoechst, bg_label=0))
ax.axis('off')

First, add these to the initial candidate pool. Deal with the large objects: search for bright spots (apoptotic nuclei) in the mask that only has large objects that were filtered out. The approach we take here is to "break up" large objects into smaller chunks and these can be further prefiltered by intensity

In [ ]:
big = np.where(~upper)[0]+1

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(label2rgb(segm*np.isin(segm, big), image=hoechst, bg_label=0))
ax.axis('off')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(hoechst*np.isin(segm, big))
plt.axis('off')

In [ ]:
from skimage.morphology import white_tophat
img_tophat = white_tophat(hoechst*np.isin(segm, big), disk(25))
img_tophat_th = threshold_img(img_tophat, method='yen')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_tophat_th)
plt.axis('off')

In [ ]:
from skimage.morphology import remove_small_objects

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(remove_small_objects(threshold_img(img_tophat, method='yen', binary=True),
                                min_size=500))
plt.axis('off')

<a id="label"></a> 
## Image labelling


<a id="trainset"></a> 
## Training and test set generation


<a id="randomforest"></a> 
## Random forest classifier

<a id="featureselect"></a> 
## Parameter tuning and feature selection

<a id="comparison"></a> 
## Comparison with other classifiers